In [1]:
%matplotlib widget
%config InlineBackend.figure_format = 'svg'

In [2]:
import addict
import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from importlib import reload
from tqdm.notebook import tqdm

import celeri
celeri = reload(celeri)

plt.rcParams['text.usetex'] = False # Plotting the global model is much much faster with tex fonts turned off

In [3]:
command_file_name = "./data/western_north_america/basic_command.json"
command, segment, block, meshes, station, mogi, sar = celeri.read_data(command_file_name)
station = celeri.process_station(station, command)
operators = addict.Dict()
operators.tri_station = celeri.get_tri_station_operator_okada(meshes, station, command)
celeri.get_all_mesh_smoothing_matrices(meshes)

# Eliminate matrix entries for vertical displacments and tensile slip
tde_matrix = copy.deepcopy(operators.tri_station)
tde_matrix = np.delete(tde_matrix, np.arange(2, tde_matrix.shape[0], 3), axis=0)
tde_matrix = np.delete(tde_matrix, np.arange(2, tde_matrix.shape[1], 3), axis=1)

Calculating cutde partials for triangles:   0%|          | 0/1841 [00:00<?, ?it/s]

In [4]:
def plot_slip_distributions(mesh, slip_distribution_input, slip_distribution_estimated, suptitle_string):
    marker_size = 2.0
    plt.figure(figsize=(12, 7))

    plt.subplot(2, 2, 1)
    plt.title("input strike-slip")
    plt.scatter(mesh.centroids[:, 0], mesh.centroids[:, 1], s=marker_size, c=slip_distribution_input[0::2], alpha=0.5)
    plt.colorbar()
    plt.gca().set_aspect("equal", adjustable="box")
    plt.xlim([230.0, 245.0])
    plt.ylim([37.5, 52.5])

    plt.subplot(2, 2, 2)
    plt.title("input dip-slip")
    plt.scatter(mesh.centroids[:, 0], mesh.centroids[:, 1], s=marker_size, c=slip_distribution_input[1::2], alpha=0.5)
    plt.colorbar()
    plt.gca().set_aspect("equal", adjustable="box")
    plt.xlim([230.0, 245.0])
    plt.ylim([37.5, 52.5])

    plt.subplot(2, 2, 3)
    plt.title("estimated strike-slip")
    plt.scatter(mesh.centroids[:, 0], mesh.centroids[:, 1], s=marker_size, c=slip_distribution_estimated[0::2], alpha=0.5)
    plt.colorbar()
    plt.gca().set_aspect("equal", adjustable="box")
    plt.xlim([230.0, 245.0])
    plt.ylim([37.5, 52.5])

    plt.subplot(2, 2, 4)
    plt.title("estimated dip-slip")
    plt.scatter(mesh.centroids[:, 0], mesh.centroids[:, 1], s=marker_size, c=slip_distribution_estimated[1::2], alpha=0.5)
    plt.colorbar()
    plt.gca().set_aspect("equal", adjustable="box")
    plt.xlim([230.0, 245.0])
    plt.ylim([37.5, 52.5])

    plt.suptitle(suptitle_string)
    plt.show()


def get_synthetic_displacements(mesh, tri_linear_operator):
    """
    Prescribe dip-slip in a Gaussian pattern.
    """
    tri_centroid_to_mesh_lon = mesh.centroids[:, 0] - np.mean(mesh.centroids[:, 0])
    tri_centroid_to_mesh_lat = mesh.centroids[:, 1] - np.mean(mesh.centroids[:, 1])

    # Hardcoded northern Cascadia example that Jack suggested.
    # tri_centroid_to_mesh_lon = mesh.centroids[:, 0] - 234.5
    # tri_centroid_to_mesh_lat = mesh.centroids[:, 1] - 48.5

    tri_centroid_to_mesh_centroid_distance = np.sqrt(tri_centroid_to_mesh_lon ** 2 + tri_centroid_to_mesh_lat ** 2)
    dip_slip_distribution = np.exp(-(tri_centroid_to_mesh_centroid_distance / 1.0) ** 2.0)
    slip_distribution = np.zeros(2 * dip_slip_distribution.size)
    slip_distribution[1::2] = dip_slip_distribution # Dip slip only
    synthetic_displacements = tri_linear_operator @ slip_distribution
    return slip_distribution, synthetic_displacements


# Generate Guassian slip source and synthetic displacements
slip_distribution, synthetic_displacements = get_synthetic_displacements(meshes[0], tde_matrix)

In [5]:
# Slip estimation with np.linalg.lstsq
estimated_slip_distribution = np.linalg.lstsq(tde_matrix, synthetic_displacements, rcond=None)
plot_slip_distributions(meshes[0], slip_distribution, estimated_slip_distribution[0], suptitle_string="np.linalg.lstsq - no smoothing")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
# Slip estimation with direct inverse no smoothing
estimated_slip_distribution = np.linalg.inv(tde_matrix.T @ tde_matrix) @ tde_matrix.T @ synthetic_displacements
plot_slip_distributions(meshes[0], slip_distribution, estimated_slip_distribution, suptitle_string="Direct inverse - no smoothing")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
# Slip estimation with direct inverse and smoothing matrix
smoothing_matrix = meshes[0].smoothing_matrix.toarray()
smoothing_matrix = np.delete(smoothing_matrix, np.arange(2, smoothing_matrix.shape[0], 3), axis=0)
smoothing_matrix = np.delete(smoothing_matrix, np.arange(2, smoothing_matrix.shape[1], 3), axis=1)
smoothing_matrix = meshes[0].smoothing_weight * smoothing_matrix # Weight smoothing matrix
tde_and_smoothing_matrix = np.vstack((tde_matrix, smoothing_matrix))
synthetic_displacements_with_smoothing = np.hstack((synthetic_displacements, np.zeros(smoothing_matrix.shape[0]).T))
estimated_slip_distribution = np.linalg.inv(tde_and_smoothing_matrix.T @ tde_and_smoothing_matrix) @ tde_and_smoothing_matrix.T @ synthetic_displacements_with_smoothing
plot_slip_distributions(meshes[0], slip_distribution, estimated_slip_distribution, suptitle_string="Direct inverse - with smoothing")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
# Slip estimation with truncated SVD
U, S, V_transpose = np.linalg.svd(tde_matrix)

In [9]:
# The general inverse is:
print("tde_matrix.shape :", tde_matrix.shape)
print("V_transpose.shape :", V_transpose.shape)
print("S.shape :", S.shape)
print("U.shape :", U.shape)
print("synthetic_displacements.shape :", synthetic_displacements.shape)

S_matrix = np.zeros_like(tde_matrix.T)
S_matrix_diag_index = np.arange(0, S.size)
S_matrix[S_matrix_diag_index, S_matrix_diag_index] = 1 / S
print("S_matrix.shape :", S_matrix.shape)

n_eigenvectors = 50
eigenvector_weights = np.linalg.pinv(tde_matrix @ V_transpose[:, 0:n_eigenvectors]) @ synthetic_displacements
estimated_slip_truncated_svd = V_transpose[:, 0:n_eigenvectors] @ eigenvector_weights
plot_slip_distributions(meshes[0], slip_distribution, estimated_slip_truncated_svd, suptitle_string="Truncated SVD - Right eigenvectors only")

# Try full SVD
estimated_slip_full_svd = V_transpose @ S_matrix @ U.T @ synthetic_displacements
plot_slip_distributions(meshes[0], slip_distribution, estimated_slip_truncated_svd, suptitle_string="Full SVD")



# Try the singular value approach for old times sake:
# https://stackoverflow.com/questions/18452633/how-do-i-associate-which-singular-value-corresponds-to-what-entry

tde_matrix.shape : (3372, 3682)
V_transpose.shape : (3682, 3682)
S.shape : (3372,)
U.shape : (3372, 3372)
synthetic_displacements.shape : (3372,)
S_matrix.shape : (3682, 3372)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
# Fall 2020 smoothing matrix SVD approach
# I think we only need the SVD of sparse smoothing matrix (I think there's scipy function for this)
from scipy.sparse.linalg import svds
n_eigenvalues = 20
U, S, VT = scipy.sparse.linalg.svds(meshes[0].smoothing_matrix, k=n_eigenvalues)

print(U.shape)
print(S.shape)
print(VT.shape)

plt.figure()
plt.plot(S, "r+")
plt.show()



(5523, 20)
(20,)
(20, 5523)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …